# In this notebook we generate the six functions that will be consulted in the API.

## Modules needed

In [3]:
import pandas as pd
import numpy as np

## User Data function

This function takes as input the user id and returns the amount of money spent by the user, the recommendation percentage and the number of games bought.

In [5]:
def userdata(user_id: str):

    ## Step 1. Import General User Items dataframe and filter using user id input
    path = 'TransformedData/'
    fn_userItems = path + 'df_userItems.csv'
    df_userItems = pd.read_csv(fn_userItems)
    df_userItems_i = df_userItems[df_userItems['user_id']==user_id]

    # Check if user id is valid
    if len(df_userItems_i) == 0:
        cantidad = 0
        cantidadItems = 0
    else:
        ## Step 2. Save number of items
        cantidadItems = df_userItems_i['items_count'].values[0]

        # Check if the user has items
        if cantidadItems == 0:
            cantidad = 0
        else:
            ## Step 3. Import Specific User Items dataframe
            steam_id = df_userItems_i['steam_id'].values[0]
            pathItems = path + 'ItemsData/'
            fni = pathItems + 'itemsData_' + str(steam_id) + '.csv'
            dfi = pd.read_csv(fni)

            ## Step 4. Import Steam Games Dataframe
            fn_steamGames = path + 'df_steamGames.csv'
            df_steamGames = pd.read_csv(fn_steamGames)

            ## Step 5. Calculate money spent iterating over item ids and looking for them in Steam Games Dataframe
            I_IDS = dfi['item_id'].values
            cantidad = 0
            for i in I_IDS:
                try:
                    p = df_steamGames[df_steamGames['id']==i]['price'].values[0]
                except:
                    p = 0 # If game is not found, assign price of 0
                cantidad += p
        
    ## Step 6. Import General User Reviews Data
    fnReviews = path + 'df_reviews.csv'
    dfReviews = pd.read_csv(fnReviews)
    dfReviews_i = dfReviews[dfReviews['user_id']==user_id]

    # Check if user id is valid
    if len(dfReviews_i) == 0:
        r = 0
    else:
        steam_id = dfReviews_i['steam_id'].values[0]
        ## Step 7. Import Specific User Reviews Data
        fnReviews_i = path + 'ReviewsData/revData_' + str(steam_id) + '.csv'
        try:
            dfReviews_i = pd.read_csv(fnReviews_i)

            ## Step 8. Calculate recommendation percentage
            n = len(dfReviews_i)
            s = sum(dfReviews_i['recommend'])
            r = n/s*100
        except:
            r=0
    
    return(cantidad,cantidadItems,r)

In [6]:
userdata('Riot-Punch')

(3427.119999999979, 328, 0)

## Count Reviews Function

This function takes as input two dates and returns the amount of users that made reviews within those dates and recommendation percentage.

In [1]:
def countreviews(date1: str, date2: str):
    

SyntaxError: incomplete input (261195287.py, line 2)

In [6]:
import pandas as pd
fn_reviews = 'TransformedData/df_reviews.csv'
df_reviews = pd.read_csv(fn_reviews)
SIDs = df_reviews['steam_id'].values
for s in SIDs[0:1]:
    fn_reviews_i = 'FeatEngData/ReviewsData/revData_' + str(s) + '.csv'
    df_reviews_i = pd.read_csv(fn_reviews_i)
    print(df_reviews_i)


   funny                    posted  last_edited  item_id         helpful  \
0    NaN  Posted November 5, 2011.          NaN     1250  No ratings yet   
1    NaN     Posted July 15, 2011.          NaN    22200  No ratings yet   
2    NaN    Posted April 21, 2011.          NaN    43110  No ratings yet   

   recommend  sentiment_analysis  
0       True                   2  
1       True                   2  
2       True                   1  


## Genre Function

This function takes as input the genre and returns the rank based on `PlayTimeForever` column

In [4]:
def genre(genre: str):
    path = 'APIData/'
    fn_genresRank = path + 'genresRank.csv'
    df_genresRank = pd.read_csv(fn_genresRank)

    genreRow = df_genresRank[df_genresRank['genre'] == genre]
    if len(genreRow) == 0:
        return('Not a genre (case sensitive)')
    else:
        return {"rank": genreRow.index[0] + 1}

In [8]:
genre('Action')

{'rank': 1}

## User for Genre Function

This function takes as input the genre and returns the top 5 user rank based on `PlayTimeForever` column, with his URL and user_id

In [47]:
def userforgenre(genre: str):
    path = 'APIData/GenresData/'
    fn_genreRank = path + 'genreData_' + genre + '.csv'
    try:
        df_genreRank = pd.read_csv(fn_genreRank)
        top5 = df_genreRank.loc[0:4,'steam_id'].values
        fn_userItems = 'APIData/df_userItems.csv'
        df_userItems = pd.read_csv(fn_userItems)

        df_top5 = df_userItems[df_userItems['steam_id'].isin(top5)].drop(columns=['items_count','steam_id'])
        
        if len(df_genreRank) == 0:
            return('No Playing time')
        else:
            return {"top5": df_top5}
    except:
        return('Not a genre (case sensitive)')

In [48]:
userforgenre('Dark')

{'top5':                  user_id                                           user_url
 4915    1001101001110101      http://steamcommunity.com/id/1001101001110101
 10898            otheldo               http://steamcommunity.com/id/otheldo
 41509  76561198018684159  http://steamcommunity.com/profiles/76561198018...
 43928      MaeveOfSwords         http://steamcommunity.com/id/MaeveOfSwords
 75278  76561198084332433  http://steamcommunity.com/profiles/76561198084...}

## Pruebas

In [26]:
path = 'TransformedData/'
fn_userItems = path + 'df_userItems.csv'
df_userItems = pd.read_csv(fn_userItems)
df_userItems

,user_id,items_count,steam_id,user_url
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr
...,...,...,...,...
87621,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...
87622,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...
87623,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...
87624,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [37]:
user_id = '76561197970982479'

path = 'TransformedData/'
fn_userItems = path + 'df_userItems.csv'
df_userItems = pd.read_csv(fn_userItems)
df_userItems_i = df_userItems[df_userItems['user_id']==user_id]

cantidadItems = df_userItems_i['items_count'].values[0]

steam_id = df_userItems_i['steam_id'].values[0]
pathItems = path + 'ItemsData/'
fni = pathItems + 'itemsData_' + str(steam_id) + '.csv'
dfi = pd.read_csv(fni)

In [34]:
fnGames = 'TransformedData/df_steamGames.csv'
dfGames = pd.read_csv(fnGames)
sum(dfGames['id'] == 413851)

0

In [81]:
steam_id = 76561197963445855
fnReviews = 'TransformedData/df_reviews.csv'
dfReviews = pd.read_csv(fnReviews)
dfReviews

,user_id,user_url,steam_id
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,76561197970982479
1,js41637,http://steamcommunity.com/id/js41637,76561198035864385
2,evcentric,http://steamcommunity.com/id/evcentric,76561198007712555
3,doctr,http://steamcommunity.com/id/doctr,76561198002099482
4,maplemage,http://steamcommunity.com/id/maplemage,76561198026584251
...,...,...,...
25481,76561198306599751,http://steamcommunity.com/profiles/76561198306...,76561198306599751
25482,Ghoustik,http://steamcommunity.com/id/Ghoustik,76561198308105615
25483,76561198310819422,http://steamcommunity.com/profiles/76561198310...,76561198310819422
25484,76561198312638244,http://steamcommunity.com/profiles/76561198312...,76561198312638244


In [82]:
dfReviews[dfReviews['steam_id'] == steam_id]

,user_id,user_url,steam_id


In [83]:
fnReviews_i = 'TransformedData/ReviewsData/revData_' + str(steam_id) + '.csv'
dfReviews_i = pd.read_csv(fnReviews_i)
n = len(dfReviews_i)
s = sum(dfReviews_i['recommend'])
print(s/n*100)


FileNotFoundError: [Errno 2] No such file or directory: 'TransformedData/ReviewsData/revData_76561197963445855.csv'

In [84]:
dfReviews_i['recommend']

0    True
1    True
2    True
Name: recommend, dtype: bool